# Irreducible representations of a finite abelian group over a finite field

* Author: Thomas Breuer
* Version: OSCAR version 1.5
* Last modified: October 13, 2025

Let $G$ be a finite abelian group, and let $F$ be a finite field.
A forthcoming paper describes how one can compute the degrees of the $F$-irreducible representations of $G$ and their multiplicities, and sketches how to compute the $F$-irreducible representations of $G$

In the following, we show [OSCAR](https://oscar.computeralgebra.de) code for addressing these two tasks.

In [1]:
using Oscar

  ___   ___   ___    _    ____
 / _ \ / __\ / __\  / \  |  _ \  | Combining and extending ANTIC, GAP,
| |_| |\__ \| |__  / ^ \ |  ´ /  | Polymake and Singular
 \___/ \___/ \___//_/ \_\|_|\_\  | Type "?Oscar" for more information
o--------o-----o-----o--------o  | Documentation: https://docs.oscar-system.org
  S Y M B O L I C   T O O L S    | 1.5.0


We start with the function that takes the vector of abelian invariants `invariants` of $G$ and the order `q` of $F$,
and computes the multiset of $F$-irreducible degrees of $G$.

Following the paper, we first observe that each $F$-irreducible degree $n$ of $G$ has the form $n = ord(q \bmod d)$, where $d$ is a divisor of the exponent $e$ of $G$.
Next we see that the number of $F$-irreducibles of degree $n$ is equal to $\frac{1}{n} \sum_{d \in D_n} |I_d(G)|$, where $D_n$ is the set of those divisors $d$ of $e$ such that $ord(q \bmod d) = n$ holds, and $I_d(G)$ is the set of those absolutely irreducible representations of $G$ in characteristic dividing `q` whose image has order $d$.

The key step is thus the computation of the cardinalities $|I_d(G)|$, which can be reduced to the case that $d$ is a prime power.

In [2]:
function irreducible_degrees(invariants::Vector{<:Oscar.IntegerUnion}, q::Oscar.IntegerUnion)
  # `q = l^f` for a prime `l`.
  flag, f, l = is_prime_power_with_data(ZZ(q))
  @req flag "q must be a prime power"

  # Switch to prime power invariants, remove the `l`-parts.
  ab_inv = abelian_invariants(ZZRingElem, invariants)
  ab_inv = [remove(x, l)[2] for x in ab_inv]

  # Distribute `ab_inv` to the prime divisors `p` of the exponent `e`.
  e = lcm(ab_inv)
  D = Dict((p, ZZRingElem[]) for p in prime_divisors(e))
  for a in ab_inv
    flag, ff, p = is_prime_power_with_data(a)
    flag && push!(D[p], ff)
  end

  # Precompute the cardinalities $|I_{p^i}(G_p)|$.
  I = Dict{ZZRingElem, ZZRingElem}()
  for p in prime_divisors(e)
    as = D[p]
    valp = p^length(as)
    I[p] = (p == 2 ? valp : (valp - 1))
    for i in 2:maximum(as)
      valnext = prod([p^min(x, i) for x in as]; init = 1)
      I[p^i] = valnext - valp
      valp = valnext
    end
  end

  # Compute the degrees `n` and their multiplicities `m`.
  res = MSet{ZZRingElem}()
  for d in (isodd(e) ? divisors(e) : 2 * divisors(div(e, 2)))
    k = (d == 1 ? 1 : modord(l, d))
    g = gcd(f, k)
    num = prod([I[x[1]^x[2]] for x in factor(d)]; init = 1)
    n = div(k, g) 
    m = Int(div(g*num, k))
    push!(res, n, m)
  end

  return res
end;

We compute some examples.

In [3]:
irreducible_degrees([3, 9, 9, 27], 2)

MSet{ZZRingElem} with 635 elements:
  6  : 351
  2  : 40
  18 : 243
  1

In [4]:
irreducible_degrees([3, 9, 9, 27], 8)

MSet{ZZRingElem} with 1823 elements:
  6 : 729
  2 : 1093
  1

In [5]:
irreducible_degrees([3, 9, 9, 27], 64)

MSet{ZZRingElem} with 3645 elements:
  3 : 1458
  1 : 2187

In [6]:
irreducible_degrees([5, 9], 2^1)

MSet{ZZRingElem} with 8 elements:
  4  : 3
  6
  2
  12 : 2
  1

In [7]:
irreducible_degrees([5, 9], 2^2)

MSet{ZZRingElem} with 15 elements:
  6 : 4
  2 : 6
  3 : 2
  1 : 3

In [8]:
irreducible_degrees([5, 9], 2^3)

MSet{ZZRingElem} with 14 elements:
  4 : 9
  2 : 4
  1

In [9]:
irreducible_degrees([5, 9], 2^4)

MSet{ZZRingElem} with 25 elements:
  3 : 10
  1 : 15

In [10]:
irreducible_degrees([5, 9], 2^6)

MSet{ZZRingElem} with 27 elements:
  2 : 18
  1 : 9

In [11]:
irreducible_degrees([5, 9], 2^12)

MSet{ZZRingElem} with 45 elements:
  1 : 45

Next we show a function that computes the $F$-irreducible representations of the group $G$.
The idea is that, for $F$ of characteristic $p$, the absolutely irreducible representations of the Hall $p'$-subgroup of $G$ are described by the Oscar function `dual`, and that each $F$-irreducible representation arises as a Galois sum of absolutely irreducible representations.

Since the image of each irreducible representation of an abelian group is cyclic, it suffices to write down a matrix $M$ over $F$ that generates the image, and to use the available implementation of the dual group to compute from $M$ the images of the generators of $G$.

In [12]:
function irreducible_representations_abelian(G::Oscar.GAPGroup, F::FinField)
  @assert is_abelian(G)
  q = order(F)
  p = characteristic(F)
  k, _ = remove(q, p)  # `F` has order `p^k`

  if mod(order(G), p) != 0
    GG = G
  else
    # Replace `G` by its Hall p'-subgroup,
    # such that the generators correspond to those of `G`.
    # (In the end, we will construct modules,
    # there is no need to create an epimorphism to the largest p'-factor.)
    GG = sub(G, [g^ppio(order(g), p)[1] for g in gens(G)])[1]
  end

  mA = isomorphism(Hecke.FinGenAbGroup, GG) # GG -> A
  A = codomain(mA)
  R, mR = dual(A)

  res = GModule{typeof(G), AbstractAlgebra.Generic.FreeModule{elem_type(F)}}[]
  all_elem = []  # already used
  rootinfo = Dict{ZZRingElem, Tuple}()

  for r in R
    r in all_elem && continue
    o = order(r)  # `r` needs a prim. `o`-th root in the finite field
    orblen = 1
    rr = q * r
    while rr != r
      push!(all_elem, rr)
      orblen = orblen + 1
      rr = q * rr
    end

    # `orblen` is the length of the Galois orbit over `F`.
    # Compute `L, K, z` where
    # `L` is the smallest field in char. `p` that contains a prim. `o`-th root,
    # `K` is the intersection of `L` and `F`,
    # and `z` is an element of order `o` in `L`.
    # These values depend only on `o`.
    # (The irred. `F`-repres. of `G` live already over `K`,
    # we work over `K` and later write the matrices over `F`.)
    L, K, z = get(rootinfo, o) do
      m = modord(p, o) # Note: `orblen == modord(q, o)` divides `m`
      L = GF(p, m)
      return L,
             GF(p, gcd(k, m)),
             Oscar.DiscLog.generator(L)^divexact(p^m-1, o)
    end

    mp = embed(K, L)
    mat = companion_matrix(minpoly(z, mp))  # matrix over K
    emb = embed(K, F)  # we want to write the matrices over F
    f = mR(r)
    M = free_module(F, orblen)

    function conv(r)
      e = r.elt::QQFieldElem
      return map_entries(emb, mat^Int(divexact(o, denominator(e))*numerator(e)))
    end

    push!(res, gmodule(M, G, [hom(M, M, conv(f(mA(g)))) for g = gens(GG)]))
  end
  return res
end;

Again, we show some examples.

In [13]:
G = abelian_group(PcGroup, [3, 3, 5]);
reps = irreducible_representations_abelian(G, GF(2));
multiset(map(dim, reps))

MSet{Int64} with 14 elements:
  4 : 9
  2 : 4
  1

In [14]:
reps = irreducible_representations_abelian(G, GF(2, 2));
multiset(map(dim, reps))

MSet{Int64} with 27 elements:
  2 : 18
  1 : 9